In [8]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports
import GEMS_TCO
from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2

In [12]:
from pathlib import Path
# Assuming 'config' and 'load_data' class are defined and imported elsewhere

# --- Parameters derived from your framework ---
v: float = 0.5
space: List[str] = ['4', '4']
days: List[str] = ['0', '31']
mm_cond_number: int = 20
# --- End of framework parameters ---

lat_lon_resolution = [int(s) for s in space]
days_s_e = [int(d) for d in days]
days_list = list(range(days_s_e[0], days_s_e[1]))

# These values were not in the framework, so they remain as set in your snippet
years = ['2024']
month_range = [7] 

# Assuming 'config' is available in your environment
output_path = input_path = Path(config.mac_estimates_day_path)

## load ozone data from amarel
data_load_instance = load_data2(config.mac_data_load_path)

# Call the function using the variables from the framework
df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      # <-- Add this
lon_range=[123.0, 133.0]   # <-- Add this
)

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


In [13]:
df_day_aggregated_list = []
df_day_map_list = []
for i in range(31):
    idx_for_datamap = [i*8, (i+1)*8]
    cur_map, cur_df = analysis_map_no_mm, agg_data_no_mm = data_load_instance.load_working_data(
    df_map, 
    idx_for_datamap, 
    ord_mm=None,  # or just omit it
    dtype=torch.float # or just omit it
)
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(df_day_aggregated_list[0].shape)


torch.Size([8960, 4])


In [14]:
instance1 = kernels.vecchia_experiment(0.5, df_day_map_list[0], df_day_aggregated_list[0], nns_map, mm_cond_number, nheads=10)

a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]
early_stop_vecc_5000= [2.8770e+01, 9.6000e-01, 1.0800e+00, 6.6789e-07, 1.1368e-06, 0.0000e+00,
        1.9100e+00]
scheduler_vecc_5000= [30.823365978489328, 1.325000558988864, 2.300043515398019, 5.191119207004892e-06, -9.752316964224805e-07, 0.0, 3.1756405728254657]

a = early_stop_vecc_5000 #  2174.9120
a = [2.9527e+01, 1.3223e+00, 2.2772e+00, 1.3499e-05, 5.9777e-06, 0.0000e+00,
        3.2343e+00] # 2108.4802
a = [30.527, 1.3223e+00, 2.2772e+00, 0, 0, 0,
        3.2343e+00] # 29.527: 2108.4802. 30.527: 2109.0693   -0.5891 difference

#a = scheduler_vecc_5000 # 2108.8091
#a = [30.2594, 0.665, 1.8981, 0.0, 0.1317, -0.0, 1.9785]
#a = [45.1402, 0.6299, 0.7308, -0.0003, -0.0151, 0.0, 7.8922]
#a = [21.7335, 1.2817, 1.5946, 0.042, -0.1241, 0.218, 4.8654]
#a = [20.453542336448137, 1.4506118600616982, 2.43096923637867, -0.03476556019978718, -0.1559262606484541, 0.1254833595232136, 3.938183829354925]
params = torch.tensor(a, dtype=torch.float64, requires_grad=True)

cov_map = instance1.cov_structure_saver(params, instance1.matern_cov_anisotropy_v05)  
instance1.vecchia_oct22( params, instance1.matern_cov_anisotropy_v05, cov_map )



tensor(20832.3250, dtype=torch.float64, grad_fn=<AddBackward0>)

v = 0.5 # smooth
mm_cond_number = 20
nheads = 20
lr = 0.01
step = 80
gamma_par = 0.9
epochs = 500

In [28]:
v = 0.5 # smooth
mm_cond_number = 20
nheads = 300
lr = 0.02
step = 100
gamma_par = 0.5
epochs = 500

# fit 8 hours of data idx_for_datamap = [i*8, (i+1)*8]

# try just fitting one hour idx_for_datamap = [i*8, (i*8+1)]

In [26]:
# ------------------------------------
# Main Optimization Loop (Updated and Corrected)
# ------------------------------------

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    #idx_for_datamap = [i*8, (i+1)*8]    
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
# ... (Data loading section unchanged and correct) ...
print(f"Data loaded for {len(df_day_map_list)} days.")

# --- 2. Run optimization loop over pre-loaded data ---
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]


    # Initial parameters (full vector 'a' is for reference)
    a = [28.75, 0.98, 1.06, 0, 0, 0, 1.890]
    #a = [21.75, 0.98, 1.56, 0, 0, 0, 1.890]
    #a = [28.303, 1.307, 1.563, 0, 0, 0, 4.769]
    
    # NEW: Define params as a list of 1-element tensors (one per parameter)
    params_list = [
        torch.tensor([val], dtype=torch.float64, requires_grad=True) for val in a
    ]

    # NEW: Define learning rates and groups
    lr_slow, lr_fast = 0.02, 0.02 # Assuming these are defined elsewhere
    slow_indices = [ 1, 2, 3, 4, 5, 6] # e.g., ranges, advection, beta, nugget
    fast_indices = [0] # e.g., sigmasq
    
    # Define Parameter Groups for the optimizer
    param_groups = [
        {'params': [params_list[idx] for idx in slow_indices], 'lr': lr_slow, 'name': 'slow_group'},
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Data loading is now done *before* the loop ---

    # Define device
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # Adjusted optimizer call: Passes the list of parameter groups
    optimizer, scheduler = model_instance.optimizer_fun(
            param_groups,     # <--- Pass the list of groups
            lr=lr,            # <--- Default LR (will be overridden by groups, but needed for function sig)
            betas=(0.9, 0.99), 
            eps=1e-8, 
            step_size=step, 
            gamma=gamma_par
        )

    # Calling the robust training loop
    out, epoch_ran = model_instance.run_vecc_scheduler_oct23(
            params_list,     # <--- Pass the list of parameter tensors
            optimizer,
            scheduler, 
            model_instance.matern_cov_anisotropy_v05, 
            epochs=epochs
        )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

Pre-loading data for all days...
Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [tensor([28.7500], dtype=torch.float64, requires_grad=True), tensor([0.9800], dtype=torch.float64, requires_grad=True), tensor([1.0600], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([1.8900], dtype=torch.float64, requires_grad=True)]
--- Epoch 1 / Loss: 2125.707079 ---
  Param 0: Value=28.7500, Grad=0.42832305446676067
  Param 1: Value=0.9800, Grad=3.7607030582832763
  Param 2: Value=1.0600, Grad=-22.898722379837636
  Param 3: Value=0.0000, Grad=-8.526512829121202e-14
  Param 4: Value=0.0000, Grad=2.4158453015843406e-13
  Param 5: Value=0.0000, Grad=0.0
  Param 6: Value=1.8900, Grad=2.638213385652388
---------------

# tmp testing just one block below

In [29]:
v = 0.5 # smooth
mm_cond_number = 20
nheads = 300
lr = 0.02
step = 100
gamma_par = 0.3
epochs = 700

# ------------------------------------
# Main Optimization Loop (Updated and Corrected)
# ------------------------------------

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    #idx_for_datamap = [i*8, (i+1)*8]    
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
# ... (Data loading section unchanged and correct) ...
print(f"Data loaded for {len(df_day_map_list)} days.")

# --- 2. Run optimization loop over pre-loaded data ---
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]


    # Initial parameters (full vector 'a' is for reference)
    a = [28.75, 0.98, 1.56, 0, 0, 0, 1.890]
    a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]
    a = [21.303, 1.307, 1.563, 0, 0, 0, 4.769]

    a = [28.303, 1.307, 1.563, 0, 0, 0, 4.769]
    # NEW: Define params as a list of 1-element tensors (one per parameter)
    params_list = [
        torch.tensor([val], dtype=torch.float64, requires_grad=True) for val in a
    ]

    # NEW: Define learning rates and groups
    lr_slow, lr_fast = 0.02, 0.02 # Assuming these are defined elsewhere
    slow_indices = [ 1, 2, 3, 4, 5, 6] # e.g., ranges, advection, beta, nugget
    fast_indices = [0] # e.g., sigmasq
    
    # Define Parameter Groups for the optimizer
    param_groups = [
        {'params': [params_list[idx] for idx in slow_indices], 'lr': lr_slow, 'name': 'slow_group'},
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Data loading is now done *before* the loop ---

    # Define device
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # Adjusted optimizer call: Passes the list of parameter groups
    optimizer, scheduler = model_instance.optimizer_fun(
            param_groups,     # <--- Pass the list of groups
            lr=lr,            # <--- Default LR (will be overridden by groups, but needed for function sig)
            betas=(0.9, 0.99), 
            eps=1e-8, 
            step_size=step, 
            gamma=gamma_par
        )

    # Calling the robust training loop
    out, epoch_ran = model_instance.run_vecc_scheduler_oct23(
            params_list,     # <--- Pass the list of parameter tensors
            optimizer,
            scheduler, 
            model_instance.matern_cov_anisotropy_v05, 
            epochs=epochs
        )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

Pre-loading data for all days...
Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [tensor([28.3030], dtype=torch.float64, requires_grad=True), tensor([1.3070], dtype=torch.float64, requires_grad=True), tensor([1.5630], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([4.7690], dtype=torch.float64, requires_grad=True)]
--- Epoch 1 / Loss: 2072.542406 ---
  Param 0: Value=28.3030, Grad=0.7997467182134974
  Param 1: Value=1.3070, Grad=-6.035422239558298
  Param 2: Value=1.5630, Grad=-12.561598310167653
  Param 3: Value=0.0000, Grad=3.659295089164516e-13
  Param 4: Value=0.0000, Grad=-1.1368683772161603e-13
  Param 5: Value=0.0000, Grad=0.0
  Param 6: Value=4.7690, Grad=5.266075690826141
----------------

# antoerh test

In [19]:
v = 0.5 # smooth
mm_cond_number = 20
nheads = 300
lr = 0.02
step = 100
gamma_par = 0.3
epochs = 500


# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    #idx_for_datamap = [i*8, (i+1)*8]    
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
# ... (Data loading section unchanged and correct) ...
print(f"Data loaded for {len(df_day_map_list)} days.")

# --- 2. Run optimization loop over pre-loaded data ---
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]


    # Initial parameters (full vector 'a' is for reference)
    a = [28.75, 0.98, 1.56, 0, 0, 0, 1.890]
    a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]
    a = [28.303, 1.307, 1.563, 0, 0, 0, 4.769]
    # NEW: Define params as a list of 1-element tensors (one per parameter)
    params_list = [
        torch.tensor([val], dtype=torch.float64, requires_grad=True) for val in a
    ]

    # NEW: Define learning rates and groups
    lr_slow, lr_fast = 0.02, 0.02 # Assuming these are defined elsewhere
    slow_indices = [ 1, 2, 3, 4, 5, 6] # e.g., ranges, advection, beta, nugget
    fast_indices = [0] # e.g., sigmasq
    
    # Define Parameter Groups for the optimizer
    param_groups = [
        {'params': [params_list[idx] for idx in slow_indices], 'lr': lr_slow, 'name': 'slow_group'},
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Data loading is now done *before* the loop ---

    # Define device
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # Adjusted optimizer call: Passes the list of parameter groups
    optimizer, scheduler = model_instance.optimizer_fun(
            param_groups,     # <--- Pass the list of groups
            lr=lr,            # <--- Default LR (will be overridden by groups, but needed for function sig)
            betas=(0.9, 0.8), 
            eps=1e-8, 
            step_size=step, 
            gamma=gamma_par
        )

    # Calling the robust training loop
    out, epoch_ran = model_instance.run_vecc_scheduler_oct23(
            params_list,     # <--- Pass the list of parameter tensors
            optimizer,
            scheduler, 
            model_instance.matern_cov_anisotropy_v05, 
            epochs=epochs
        )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

Pre-loading data for all days...
Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [tensor([28.3030], dtype=torch.float64, requires_grad=True), tensor([1.3070], dtype=torch.float64, requires_grad=True), tensor([1.5630], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([4.7690], dtype=torch.float64, requires_grad=True)]
--- Epoch 1 / Loss: 2072.542406 ---
  Param 0: Value=28.3030, Grad=0.7997467182134974
  Param 1: Value=1.3070, Grad=-6.035422239558298
  Param 2: Value=1.5630, Grad=-12.561598310167653
  Param 3: Value=0.0000, Grad=3.659295089164516e-13
  Param 4: Value=0.0000, Grad=-1.1368683772161603e-13
  Param 5: Value=0.0000, Grad=0.0
  Param 6: Value=4.7690, Grad=5.266075690826141
----------------

# thirs test step size

In [30]:
v = 0.5 # smooth
mm_cond_number = 20
nheads = 300
lr = 0.02
step = 70
gamma_par = 0.3
epochs = 700


# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    #idx_for_datamap = [i*8, (i+1)*8]    
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
# ... (Data loading section unchanged and correct) ...
print(f"Data loaded for {len(df_day_map_list)} days.")

# --- 2. Run optimization loop over pre-loaded data ---
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]


    # Initial parameters (full vector 'a' is for reference)
    a = [28.75, 0.98, 1.56, 0, 0, 0, 1.890]
    a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769]
    a = [28.303, 1.307, 1.563, 0, 0, 0, 4.769]
    # NEW: Define params as a list of 1-element tensors (one per parameter)
    params_list = [
        torch.tensor([val], dtype=torch.float64, requires_grad=True) for val in a
    ]

    # NEW: Define learning rates and groups
    lr_slow, lr_fast = 0.02, 0.02 # Assuming these are defined elsewhere
    slow_indices = [ 1, 2, 3, 4, 5, 6] # e.g., ranges, advection, beta, nugget
    fast_indices = [0] # e.g., sigmasq
    
    # Define Parameter Groups for the optimizer
    param_groups = [
        {'params': [params_list[idx] for idx in slow_indices], 'lr': lr_slow, 'name': 'slow_group'},
        {'params': [params_list[idx] for idx in fast_indices], 'lr': lr_fast, 'name': 'fast_group'}
    ]

    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params_list}')
            
    # --- Data loading is now done *before* the loop ---

    # Define device
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # Adjusted optimizer call: Passes the list of parameter groups
    optimizer, scheduler = model_instance.optimizer_fun(
            param_groups,     # <--- Pass the list of groups
            lr=lr,            # <--- Default LR (will be overridden by groups, but needed for function sig)
            betas=(0.9, 0.8), 
            eps=1e-8, 
            step_size=step, 
            gamma=gamma_par
        )

    # Calling the robust training loop
    out, epoch_ran = model_instance.run_vecc_scheduler_oct23(
            params_list,     # <--- Pass the list of parameter tensors
            optimizer,
            scheduler, 
            model_instance.matern_cov_anisotropy_v05, 
            epochs=epochs
        )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

Pre-loading data for all days...
Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [tensor([28.3030], dtype=torch.float64, requires_grad=True), tensor([1.3070], dtype=torch.float64, requires_grad=True), tensor([1.5630], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([0.], dtype=torch.float64, requires_grad=True), tensor([4.7690], dtype=torch.float64, requires_grad=True)]
--- Epoch 1 / Loss: 2072.542406 ---
  Param 0: Value=28.3030, Grad=0.7997467182134974
  Param 1: Value=1.3070, Grad=-6.035422239558298
  Param 2: Value=1.5630, Grad=-12.561598310167653
  Param 3: Value=0.0000, Grad=3.659295089164516e-13
  Param 4: Value=0.0000, Grad=-1.1368683772161603e-13
  Param 5: Value=0.0000, Grad=0.0
  Param 6: Value=4.7690, Grad=5.266075690826141
----------------

# old veresion single learning rate

In [17]:
v = 0.5 # smooth
mm_cond_number = 20
nheads = 300
lr = 0.02
step = 100
gamma_par = 0.3
epochs = 500

# ------------------------------------
# Main Optimization Loop (Updated)
# ------------------------------------

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(f"Data loaded for {len(df_day_map_list)} days.")
# print(df_day_aggregated_list[0].shape) # From reference code

# --- 2. Run optimization loop over pre-loaded data ---

# This list is now just for iterating
#days_list = range(len(df_day_map_list)) 
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]


    a = [28.75, 0.98, 1.06, 0, 0, 0, 1.890]
    params = torch.tensor(a, dtype=torch.float64, requires_grad=True)
    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params.detach().numpy()}')
            
    # --- Data loading is now done *before* the loop ---

    # We need to define the device (though we aren't passing it anymore)
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
            # device = device_str  <--- REMOVED: This was causing the TypeError
        )

    start_time = time.time()
    # Adjusted optimizer call based on expected return values (step size changed to step)
    optimizer, scheduler = model_instance.optimizer_fun_scheduler_same_lr(
        params, 
        lr=lr, 
        betas=(0.9, 0.8), 
        eps=1e-8, 
        step_size=step, # Using the 'step' variable here
        gamma=gamma_par  # Using gamma_par
    ) 

    # --- CRITICAL CORRECTION ---
    # 1. We no longer need to create a separate 'instance_map'.
    #    'model_instance' is already the correct instance.
    # 2. We do NOT pre-calculate 'cov_map'. The optimized training loop
    #    'run_vecc_may9' does this internally on each epoch
    #    to ensure the gradients are correct.
    # 3. We call 'run_vecc_may9' (the optimized loop) instead of 'run_vecc_grp9'.
    #    This version does not take 'cov_map' as an argument.
    
    # Calling the optimized 'run_vecc_may9'
    out, epoch_ran = model_instance.run_vecc_scheduler_same_lr(
        params, 
        optimizer,
        scheduler, 
        model_instance.matern_cov_anisotropy_v05, 
        epochs=epochs
    )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

print("\n--- All Days Processed ---")

Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 1250.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [28.75  0.98  1.06  0.    0.    0.    1.89]
Epoch 1, Gradients: [ 6.76930898e-01  2.84719128e+00 -2.83749606e+01 -3.55271368e-13
 -6.53699317e-13  0.00000000e+00  2.00032705e+00]
 Loss: 2105.305294117777, Parameters: [28.75  0.98  1.06  0.    0.    0.    1.89]
Epoch 11, Gradients: [ 6.14380725e-01 -1.75437339e+00 -1.88931280e+01  8.10018719e-13
  2.27373675e-13  0.00000000e+00  2.92466141e+00]
 Loss: 2108.104279627454, Parameters: [2.85498405e+01 8.29343906e-01 1.26224386e+00 1.10189496e-06
 5.16845217e-06 0.00000000e+00 1.69434411e+00]
Epoch 21, Gradients: [ 2.77650151e-01  1.03782795e+00 -1.07808718e+01  3.26849658e-13
 -4.26325641e-13  0.00000000e+00  1.05741661e+00]
 Loss: 2094.7642846757913, Parameters: [ 2.83390166e+01  8.66321677e-01  1.47686706e+00 -2.06307303e-06
  6.78750835e-06  0.00000000e+00  1.489963

fit one hour but fix time relevant parameters

In [6]:
# ------------------------------------
# Main Optimization Loop (Updated)
# ------------------------------------

# --- 1. Pre-load all data (based on reference code) ---
print("Pre-loading data for all days...")
df_day_aggregated_list = []
df_day_map_list = []
num_days_to_load = 31 # From reference code

for i in range(num_days_to_load):
    idx_for_datamap = [i*8, i*8+1]
    # Using the new load function from the reference code
    cur_map, cur_df = data_load_instance.load_working_data(
        df_map, 
        idx_for_datamap, 
        ord_mm=None,
        # Using float64 to ensure compatibility with kernels
        dtype=torch.float64 
    )
    df_day_aggregated_list.append( cur_df )
    df_day_map_list.append( cur_map )

print(f"Data loaded for {len(df_day_map_list)} days.")

# --- 2. Run optimization loop over pre-loaded data ---

# This list is now just for iterating
#days_list = range(len(df_day_map_list)) 
days_list = [0]
for day in days_list:  
    
    # Get the pre-loaded data for this day
    analysis_data_map = df_day_map_list[day]
    aggregated_data = df_day_aggregated_list[day]

    # **CHANGE**: Use a single initialization vector 'a' with fixed indices (3, 4, 5) set to 0.
    # a = [21.303, 1.307, 1.563, 0.022, -0.144, 0.198, 4.769] # Original
    a = [29.75, 0.98, 1.06, 0, 0, 0, 1.890]
    params = torch.tensor(a, dtype=torch.float64, requires_grad=True)
    
    # Calculate resolution for printing
    res_calc = (200 / lat_lon_resolution[0]) * (100 / lat_lon_resolution[0])
    print(f'\n--- Starting Day {day+1} (2024-07-{day+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n {params.detach().numpy()}')
            
    # --- Data loading is now done *before* the loop ---

    # We need to define the device (though we aren't passing it anymore)
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    model_instance = kernels.model_fitting(
            smooth = v,
            input_map = analysis_data_map,
            aggregated_data = aggregated_data,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
            # device = device_str  <--- REMOVED: This was causing the TypeError
        )

    start_time = time.time()
    
    # Adjusted optimizer call based on expected return values (step size changed to step)
    lr=0.03
    optimizer, scheduler = model_instance.optimizer_fun_scheduler_same_lr(
        params, 
        lr=lr, 
        betas=(0.9, 0.8), 
        eps=1e-8, 
        step_size=step, # Using the 'step' variable here
        gamma=gamma_par  # Using gamma_par
    ) 

    # --- CRITICAL CORRECTION: Call the new masked function ---
    out, epoch_ran = model_instance.run_vecc_scheduler_same_lr_fit_hour( # <-- Function name changed
        params, 
        optimizer,
        scheduler, 
        model_instance.matern_cov_anisotropy_v05, 
        epochs=epochs
    )
    # --- End Correction ---

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Day {day+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day+1} final results: {out}")

print("\n--- All Days Processed ---")

Pre-loading data for all days...
Data loaded for 31 days.

--- Starting Day 1 (2024-07-1) ---
Data size per day: 5000.0, smooth: 0.5
mm_cond_number: 20,
initial parameters: 
 [29.75  0.98  1.06  0.    0.    0.    1.89]
Epoch 1, Gradients (Full): [ -0.02896164  10.53635213 -13.62827347   0.           0.
   0.          -2.77344428]
 Loss: 8570.70743603638, Parameters (Full): [29.75  0.98  1.06  0.    0.    0.    1.89]
Epoch 51, Gradients (Full): [-0.07348001 -0.39344868 -0.09763837  0.          0.          0.
  0.83158837]
 Loss: 8315.909261744184, Parameters (Full): [29.83286556  1.31416734  2.30829604  0.          0.          0.
  3.3709474 ]
Epoch 101, Gradients (Full): [-0.08185817 -0.02543247 -0.01331501  0.          0.          0.
 -0.08306768]
 Loss: 8314.451279317555, Parameters (Full): [31.35687947  1.32904135  2.31022858  0.          0.          0.
  3.1419399 ]
Epoch 151, Gradients (Full): [-0.07862879 -0.0057642  -0.00551532  0.          0.          0.
 -0.01512818]
 Loss: 83